In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from keras_tqdm import TQDMNotebookCallback

import pickle

Using MXNet backend.
/anaconda/envs/py35/lib/python3.5/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
import cntk
import tensorflow
import theano


print ("Keras Version: ",keras.__version__)
print ("TensorFlow Version: ",tensorflow.__version__)
print ("CNTK Version: ",cntk.__version__)
print ("Theano Version: ",theano.__version__)

/anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/anaconda/envs/py35/lib/python3.5/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "


Keras Version:  1.2.2
TensorFlow Version:  1.3.0
CNTK Version:  2.1
Theano Version:  0.9.0


Using cuDNN version 6021 on context None
Mapped name None to device cuda: Tesla K80 (299E:00:00.0)


In [3]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#Below code changed because of MXNet only working with older version of KerasKeras
#if K.image_data_format() == 'channels_first':

if K.image_dim_ordering() == 'tf':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
#to_categorical function not available in the Keras version 1.2.2. So as a workaround 
#I am running this code in later version of Keras and storing the variables as a Step 1.
#Step 2 - For MXNet performance testing, run the code with Keras 1.2.2 and read int hese variables from the disk

#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

AttributeError: module 'keras.utils' has no attribute 'to_categorical'

In [5]:
y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [6]:
output_file='./pckl/mnist_cnn_y_train.pckl'
f = open(output_file, 'wb')
pickle.dump(y_train, f)
f.close()

output_file='./pckl/mnist_cnn_y_test.pckl'
f = open(output_file, 'wb')
pickle.dump(y_test, f)
f.close()


In [4]:
output_file='./pckl/mnist_cnn_y_train.pckl'
f = open(output_file, 'rb')
y_train = pickle.load(f)
f.close()

output_file='./pckl/mnist_cnn_y_test.pckl'
f = open(output_file, 'rb')
y_test = pickle.load(f)
f.close()

In [7]:
y_train
input_shape

(28, 28, 1)

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

TypeError: __init__() missing 2 required positional arguments: 'nb_row' and 'nb_col'

In [26]:
#Fixed for older version of Keras to work with MXNet
model = Sequential()
model.add(Conv2D(nb_filter=32,activation='relu',nb_row=28, nb_col=28, input_shape=input_shape))
model.add(Conv2D(64,  nb_row=22,nb_col=22,activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'],context=['gpu(0)'])

MXNetError: Error in operator uniform9: [22:39:47] src/operator/./convolution-inl.h:463: Check failed: dilated_ksize_x <= AddPad(dshape[3], param_.pad[1]) (28 vs. 1) kernel size exceed input

Stack trace returned 10 entries:
[bt] (0) /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x1d57cc) [0x7f1e206237cc]
[bt] (1) /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x102211a) [0x7f1e2147011a]
[bt] (2) /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/libmxnet.so(+0xf9d2d7) [0x7f1e213eb2d7]
[bt] (3) /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x2d6d621) [0x7f1e231bb621]
[bt] (4) /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x2d6ee32) [0x7f1e231bce32]
[bt] (5) /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x2d6f776) [0x7f1e231bd776]
[bt] (6) /anaconda/envs/py35/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x2d58f78) [0x7f1e231a6f78]
[bt] (7) /anaconda/envs/py35/lib/python3.5/site-p

## Tensorflow

In [5]:
print ("Tensorflow Version: ",tensorflow.__version__)
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
%timeit -r 1 -n 1 model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Tensorflow Version:  1.3.0
Keras Backend:  tensorflow
Epochs:  12



2min 3s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Test loss: 0.0272259225131
Test accuracy: 0.9909


## CNTK

In [5]:
print ("CNTK Version: ",cntk.__version__)
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
%timeit -r 1 -n 1 model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

CNTK Version:  2.1
Keras Backend:  cntk
Epochs:  12


/anaconda/envs/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input120") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))



2min 59s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Test loss: 0.0297527180951
Test accuracy: 0.9904


## Theano

In [ ]:
print ("Theano Version: ",theano.__version__)
print("Keras Backend: ", keras.backend.backend())
print("Epochs: ", epochs)
%timeit -r 5 -n 1 model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=0,validation_data=(x_test, y_test),callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])